In [9]:
import os, time, random, shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import pandas as pd

# GPU setup
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # use only one GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
print("GPU ready:", gpus)


GPU ready: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [10]:
root_path = "/kaggle/input/11-785-fall-20-homework-2-part-2"
train_dir = f"{root_path}/classification_data/train_data"
val_dir   = f"{root_path}/classification_data/val_data"

# Create smaller subset for faster training
def create_subset(src, dst, limit_per_class=60):
    os.makedirs(dst, exist_ok=True)
    for cls in os.listdir(src):
        src_cls = os.path.join(src, cls)
        dst_cls = os.path.join(dst, cls)
        if not os.path.isdir(src_cls): continue
        os.makedirs(dst_cls, exist_ok=True)
        imgs = os.listdir(src_cls)
        for img in random.sample(imgs, min(limit_per_class, len(imgs))):
            shutil.copy(os.path.join(src_cls, img), os.path.join(dst_cls, img))

if not os.path.exists("/kaggle/working/train_subset"):
    create_subset(train_dir, "/kaggle/working/train_subset", 60)
    create_subset(val_dir, "/kaggle/working/val_subset", 20)

train_dir = "/kaggle/working/train_subset"
val_dir   = "/kaggle/working/val_subset"

img_size = (160,160)
batch_size = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
).flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

num_classes = train_gen.num_classes
print("Classes:", num_classes)


Found 233470 images belonging to 4000 classes.
Found 8000 images belonging to 4000 classes.
Classes: 4000


In [ ]:
# Build EfficientNetB0
base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=img_size+(3,))

# Freeze earlier layers, fine-tune last 80
for layer in base.layers[:-80]:
    layer.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation='softmax', dtype='float32')(x)
softmax_model = Model(inputs=base.input, outputs=output)

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
softmax_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=1, restore_best_weights=True),
    ReduceLROnPlateau(patience=1, factor=0.5, min_lr=1e-6)
]

start = time.time()
history = softmax_model.fit(train_gen, validation_data=val_gen, epochs=8, callbacks=callbacks)
print("⏱ Training finished in", round((time.time()-start)/60,1), "minutes")

softmax_model.save("/kaggle/working/softmax_model.keras")
print("Saved softmax_model.keras")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8


E0000 00:00:1762780443.837220     116 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1762780443.974571     116 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


3473/7296 ━━━━━━━━━━━━━━━━━━━━ 10:58 172ms/step - accuracy: 3.6839e-04 - loss: 8.5702

In [ ]:
embedding_model = Model(inputs=softmax_model.input, outputs=softmax_model.layers[-2].output)
embedding_model.save("/kaggle/working/embedding_model.keras")
print("Saved embedding_model.keras")

In [ ]:
pairs = pd.read_csv(f"{root_path}/verification_pairs_val.txt", sep=" ", header=None, names=["img1","img2","label"])

def preprocess(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
    arr = tf.keras.preprocessing.image.img_to_array(img)/255.
    return np.expand_dims(arr, axis=0)

def embed_image(path):
    return embedding_model.predict(preprocess(path), verbose=0).squeeze()

def cosine_sim(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

def euclid_sim(a,b):
    return -np.linalg.norm(a-b)

cos_scores, euc_scores, labels = [], [], []

for _, r in pairs.iterrows():
    img1 = os.path.join(root_path, r.img1)
    img2 = os.path.join(root_path, r.img2)
    emb1, emb2 = embed_image(img1), embed_image(img2)
    cos_scores.append(cosine_sim(emb1, emb2))
    euc_scores.append(euclid_sim(emb1, emb2))
    labels.append(r.label)

auc_cos = roc_auc_score(labels, cos_scores)
auc_euc = roc_auc_score(labels, euc_scores)
print("AUC (Cosine):", round(auc_cos,4))
print("AUC (Euclidean):", round(auc_euc,4))

# Plot ROC Curve
fpr, tpr, _ = roc_curve(labels, cos_scores)
plt.plot(fpr, tpr, label=f"Cosine AUC={auc_cos:.4f}")
plt.plot([0,1], [0,1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Face Verification")
plt.legend()
plt.show()

In [ ]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    total_leng = y_pred.shape[-1] // 3
    a, p, n = y_pred[:, :total_leng], y_pred[:, total_leng:2*total_leng], y_pred[:, 2*total_leng:]
    pos_dist = tf.reduce_sum(tf.square(a-p), axis=1)
    neg_dist = tf.reduce_sum(tf.square(a-n), axis=1)
    return tf.reduce_mean(tf.maximum(pos_dist - neg_dist + alpha, 0.0))

input_a = Input(shape=img_size+(3,))
input_p = Input(shape=img_size+(3,))
input_n = Input(shape=img_size+(3,))

emb_a = embedding_model(input_a)
emb_p = embedding_model(input_p)
emb_n = embedding_model(input_n)
merged = Concatenate(axis=1)([emb_a, emb_p, emb_n])

triplet_model = Model([input_a, input_p, input_n], merged)
triplet_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss=triplet_loss)
triplet_model.save("/kaggle/working/triplet_model.keras")
print("Saved triplet_model.keras (structure ready for fine-tuning)")

In [ ]:
pairs_test = pd.read_csv(f"{root_path}/verification_pairs_test.txt", sep=" ", header=None, names=["img1","img2"])
scores_test = []

for _, r in pairs_test.iterrows():
    img1 = os.path.join(root_path, r.img1)
    img2 = os.path.join(root_path, r.img2)
    emb1, emb2 = embed_image(img1), embed_image(img2)
    scores_test.append(cosine_sim(emb1, emb2))

submission = pd.DataFrame({
    "pair": [f"{r.img1} {r.img2}" for _, r in pairs_test.iterrows()],
    "score": scores_test
})
submission.to_csv("/kaggle/working/hw2p2_submission.csv", index=False)
print("Generated submission file: hw2p2_submission.csv")